<a href="https://colab.research.google.com/github/kangwonlee/2018pycpp/blob/colab-buttons/50.under-the-hood/22.working_with_bits_operators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Working with bits



* To utilize certain features of microcontrollers, often times we need to turn on or off certain bits of certain memory location.



* The pointer of C/C++ is a suitable tool for this type of task.



* However, the smallest unit of memory that we can change is a byte; hence we may need to first read the current status of the particular byte, change that one bit, and then write the *bit pattern* to that byte.



## 16 bit color example



* IBM's [XGA](https://en.wikipedia.org/wiki/Graphics_display_resolution) graphic card standard in 1990s was the first with 1024 &times; 768 resolution. Its 640 &times; 480 mode could use 16 bit [*high color*](https://en.wikipedia.org/wiki/High_color).



* To represent red, green, and blue colors with 15 bits, 5 bits per each would be the natural choice.



* With 16 bits, 1 more bit for green; hence red 5, green 6, and blue 5 bits.



In [ ]:
# https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook
# http://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Rich%20Output.ipynb

import IPython.display as disp

# number of bits
n = 16
nr = 5
ng = 6
nb = 5

disp.display(
    disp.Markdown(
        '\n'.join(
            [
                ' | '.join(str(k) for k in range(n-1, 0-1, -1)),
                '|'.join(':---:' for k in range(1, n+1)),
                ' | '.join(['`b`']*nb + ['`g`']*ng + ['`r`']*nr),
            ], # Prepare a list of table rows
        ) # Join all rows with new line characters in between
    ) # Create a markdown object with the string for table
) # Present the markdown table



* But why 1 more bit for green?
* Why three color channels from the beginning?



* Anyways, let's see if we can pack three colors in 16 bits.



### Revisiting bitwise operators



* Both C/C++ and python has a number of [bitwise operators](https://en.wikipedia.org/wiki/Bitwise_operation).



| operator | python | C/C++ |
|:-----:|:-----:|:-----:|
| `<<` | `1 << 5` |  `1 << 5` |
| `>>` | `int('10000', base=2) & >> 4` |  `0x10 >> 4` |
| `&` | `int('11101011', base=2) & int('11111', base=2)` |  `235 << 31` |
| <code>&vert;</code> | 0 <code>&vert; (1 << 7)</code> |  `0 << 128` |
| `&=` | `a = 10; a &= int('0111', base=2)` |  `char a = 10; a &= 0b0111;` |



#### 16bit color using bitwise operator



* Let's try to implement 16bit color using the bitwise operators.



In [ ]:
%%writefile bitwise_16bit_color.cpp
#include <bitset>
#include <cstdint>
#include <cstdlib>
#include <iomanip>
#include <iostream>


int32_t main(const int32_t argn, const char * argv[]){

    // number of examples
    const int32_t n = 10;

    // table header
    std::cout << "| `r` | `g` | `b` | `hex` |" << '\n';
    std::cout << "|:---:|:---:|:---:|:-----:|" << '\n';

    uint32_t i = 0;
    // example loop
    for(i = 0; n > i; ++i){
        // variables for r g b colors and index
        uint32_t r = rand() % (1 << 5);
        uint32_t g = rand() % (1 << 6);
        uint32_t b = rand() % (1 << 5);
        uint32_t hex = 0;

        // set red value from bit 0 ~ 4
        hex |= (r & 0x1F);
        
        // set green value from bit 5 ~ 10
        hex |= ((g & 0x3F) << 5);
        
        // set blue value from bit 11 ~ 15
        hex |= ((b & 0x1F) << 11);
        
        // How hex is supposed to have 16bit color value

        // bit patterns of color values
        // https://stackoverflow.com/questions/7349689
        std::bitset<5> b_r(r);
        std::bitset<6> b_g(g);
        std::bitset<6> b_b(b);

        // bit pattern of the 16bit integer
        std::bitset<16> b_hex(hex);

        // print this example as a row
        std::cout << "| " << std::hex << std::setw(6) << b_r
                    << " | " << std::setw(6) << b_g
                    << " | " << std::setw(6) << b_b
                    << " | " << std::setw(16) << b_hex
                    << " |\n";
    }

}



In [ ]:
%%bash
# The directive above would run following as bash commands

# Detect OS type because OSX may need different options
# https://stackoverflow.com/questions/3466166/how-to-check-if-running-in-cygwin-mac-or-linux/18790824

# obtain system information
unameOut="$(uname -s)"

# detect system type
case "${unameOut}" in
    Linux*)     machine=Linux;;
    Darwin*)    machine=Mac;;
    CYGWIN*)    machine=Cygwin;;
    MINGW*)     machine=MinGw;;
    *)          machine="UNKNOWN:${unameOut}"
esac

if [ $machine == "Linux" ]; then
    # build command for Linux
    g++ -Wall -g bitwise_16bit_color.cpp -o ./bitwise_16bit_color -Wa,-adhln=bitwise_16bit_color.s
elif [ "Mac" == $machine ]; then
    # build command for OSX
    # https://stackoverflow.com/questions/10990018/
    clang++ -S -mllvm --x86-asm-syntax=intel bitwise_16bit_color.cpp
    clang++ -Wall -g bitwise_16bit_color.cpp -o bitwise_16bit_color
else
    # Otherwise
    g++ -Wall -g bitwise_16bit_color.cpp -o ./bitwise_16bit_color.s -S
    g++ -Wall -g bitwise_16bit_color.cpp -o ./bitwise_16bit_color
fi



In [ ]:
# https://stackoverflow.com/questions/4760215/running-shell-command-from-python-and-capturing-the-output
# https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook
import subprocess
import IPython.display as disp

# Run executable while capturing output
result = subprocess.run(['./bitwise_16bit_color'], stdout=subprocess.PIPE)

# present output as a markdown table
disp.display(disp.Markdown(result.stdout.decode()))



In [ ]:
%%bash
# Run bash command to delete .cpp and executable files
rm bitwise_16bit_color.cpp bitwise_16bit_color

